In [2]:
!pip install tensorflow numpy matplotlib ultralytics

In [3]:
!pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("techsash/waste-classification-data")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\kelka\.cache\kagglehub\datasets\techsash\waste-classification-data\versions\1


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(validation_split = 0.2, rescale = 1./225)

train_data = datagen.flow_from_directory(
    path, target_size = (224, 224), batch_size = 32,
    class_mode = 'binary', subset = "training"
)

val_data = datagen.flow_from_directory(
    path, target_size = (224, 224), batch_size = 32,
    class_mode = 'binary', subset = "validation"
)

Found 40124 images belonging to 1 classes.
Found 10030 images belonging to 1 classes.


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation = "relu", input_shape = (224, 224, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation = "relu"),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation = "relu"),
    layers.Dense(1, activation = "sigmoid")
])

model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

C:\Users\kelka\anaconda3\envs\opencv_env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = model.fit(train_data, validation_data = val_data, epochs = 10)

C:\Users\kelka\anaconda3\envs\opencv_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
  65/1254 ━━━━━━━━━━━━━━━━━━━━ 17:10 866ms/step - accuracy: 0.9497 - loss: 0.0519

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import os

test_o_folder = os.path.join(path, "TEST", "O")
test_r_folder = os.path.join(path, "TEST", "R")

image_name_o = "O-12568.jpg"
image_path_o = os.path.join(test_o_folder, image_name_o)

img_o = image.load_img(image_path_o, target_size = (224, 224))
img_array_o = np.expand_dims(image.img_to_array(img_o) / 255.0, axis = 0)
prediction_o = model.predict(img_array_o)[0][0]

if prediction_o > 0.5:
    print("Organic Waste 🥦")
else:
    print("Recyclable Waste ♻️")